# Klasyfikacja obrazów zawierających klucze oczkowe, śrubokręty i młotki

Dodajemy wymagane paczki nuget

In [ ]:
#r "nuget: Microsoft.ML, 1.7.0"
#r "nuget: Microsoft.ML.TensorFlow, 1.7.0"
#r "nuget: Microsoft.ML.ImageAnalytics, 1.7.0"
#r "nuget: Microsoft.ML.OnnxTransformer, 1.7.0"
#r "nuget: Microsoft.ML.OnnxTransformer, 1.7.0"
#r "nuget: Microsoft.Data.Analysis, 0.19.0"
#r "nuget: SciSharp.TensorFlow.Redist, 2.7.0"

Installed Packages Microsoft.Data.Analysis, 0.19.0 Microsoft.ML, 1.7.0 Microsoft.ML.ImageAnalytics, 1.7.0 Microsoft.ML.OnnxTransformer, 1.7.0 Microsoft.ML.TensorFlow, 1.7.0 SciSharp.TensorFlow.Redist, 2.7.0

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;
using System.IO;

Teraz dodajmy ścieżki z obrazkami i konfiguracją dla naszego projektu. 

In [ ]:
// Główny katalog z danymi używanymi w przykładzie
var _assetsPath = Path.Combine(Environment.CurrentDirectory, "assets");

// Katalog z obrazkami jakie będą używane w naszym przykładzie (do trenowania modelu i do docelowej klasyfikacji)
var _imagesFolder = Path.Combine(_assetsPath, "images");

// Katalog z obrazkami do testowania
var _testFolder = Path.Combine(_imagesFolder, "test");

// Plik z oznaczeniami obrazków (czyli czym jaki obrazek jest). Użyjemy go do trenowania modelu
var _trainTagsTsv = Path.Combine(_imagesFolder, "tags.tsv");

// Plik z oznaczeniami obrazków, którego użyjemy do testowania naszego modelu
var _testTagsTsv = Path.Combine(_imagesFolder, "test-tags.tsv");

// Uzyjemy TensorFlow Inception. Jest to model deep learning, który jest wytrenowany na ImageNet dataset.
//
// Poniżej link gdzie znajdziesz więcej informacji o inception
// https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202
var _inceptionTensorFlowModel = Path.Combine(_assetsPath, "inception", "tensorflow_inception_graph.pb");

`ImageData` będzie klasą, która służy do tagowania obrazeków (klucz, młotek, śrubokręt). Myślę,że nazwy property są wystarczająco jasne.

In [ ]:
public class ImageData
{
    [LoadColumn(0)]
    public string Path;

    [LoadColumn(1)]
    public string Label;
}

`ImagePrediction` posłuży nam do określenia tego czym dany obrazek jest.

In [ ]:
public class ImagePrediction : ImageData
{
    public float[] Score;

    public string PredictedLabelValue;
}

Zaczynamy :-)

In [ ]:
var mlContext = new MLContext();

To są ustawienia modelu. Zostawiam takie jakie były użyte w [tutorialu](https://docs.microsoft.com/en-us/dotnet/machine-learning/tutorials/image-classification).

In [ ]:
struct InceptionSettings
{
    public const int ImageHeight = 224;
    public const int ImageWidth = 224;
    public const float Mean = 117;
    public const float Scale = 1;
    public const bool ChannelsLast = true;
}

Ta funkcja posłuży nam do sklasyfikowania obrazka

In [ ]:
void ClassifyImage(MLContext mlContext, ITransformer model, string imagePath)
{
    var imageData = new ImageData()
    {
        Path = imagePath
    };

    var predictor = mlContext.Model.CreatePredictionEngine<ImageData, ImagePrediction>(model);
    var prediction = predictor.Predict(imageData);
    Console.WriteLine($"[{Path.GetFileName(imagePath)}] jest [{prediction.PredictedLabelValue}] z wynikiem [{prediction.Score.Max()}] ");
}

Ta funkcja posłuży nam do utworzenia plików `tags.tsv` oraz `test-tags.tsv`.

In [ ]:
void CreateTsvFiles()
{
    File.Delete(_trainTagsTsv);
    File.Delete(_testTagsTsv);

    using var tsvFile = new StreamWriter(_trainTagsTsv);
    using var testTsvFile = new StreamWriter(_testTagsTsv);

    foreach (var directory in Directory.GetDirectories(_imagesFolder))
    {
        var directoryName = new DirectoryInfo(directory).Name;

        if (string.Equals(directoryName, "test"))
        {
            continue;
        }
        
        foreach (var file in Directory.GetFiles(directory))
        {
            var filename = Path.GetFileName(file);
            tsvFile.WriteLine($"{filename}\t{directoryName}");
            testTsvFile.WriteLine($"{filename}\t{directoryName}");
            File.Copy(file, $"{_imagesWorkingFolder}\\{filename}", true);
        }
    }
}

Ta funkcjia posłuży nam do utworzenia modelu

In [ ]:
ITransformer GenerateModel(MLContext mlContext)
{
    var pipeline =
        mlContext.Transforms
                 .LoadImages(outputColumnName: "input", imageFolder: _imagesFolder, inputColumnName: nameof(ImageData.Path))
                 .Append(mlContext.Transforms.ResizeImages(outputColumnName: "input", imageWidth: InceptionSettings.ImageWidth, imageHeight: InceptionSettings.ImageHeight, inputColumnName: "input"))
                 .Append(mlContext.Transforms.ExtractPixels(outputColumnName: "input", interleavePixelColors: InceptionSettings.ChannelsLast, offsetImage: InceptionSettings.Mean))
                 .Append(mlContext.Model.LoadTensorFlowModel(_inceptionTensorFlowModel).ScoreTensorFlowModel(outputColumnNames: new[] { "softmax2_pre_activation" }, inputColumnNames: new[] { "input" }, addBatchDimensionInput: true))
                 .Append(mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "LabelKey", inputColumnName: "Label"))
                 .Append(mlContext.MulticlassClassification.Trainers.LbfgsMaximumEntropy(labelColumnName: "LabelKey", featureColumnName: "softmax2_pre_activation"))
                 .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabelValue", "PredictedLabel"))
                 .AppendCacheCheckpoint(mlContext);
                 
    var trainingData = mlContext.Data.LoadFromTextFile<ImageData>(path:  _trainTagsTsv, hasHeader: false);
    var model = pipeline.Fit(trainingData);
    var testData = mlContext.Data.LoadFromTextFile<ImageData>(path: _testTagsTsv, hasHeader: false);
    var predictions = model.Transform(testData);
    mlContext.Data.CreateEnumerable<ImagePrediction>(predictions, true);
    mlContext.MulticlassClassification.Evaluate(predictions, labelColumnName: "LabelKey", predictedLabelColumnName: "PredictedLabel");

    return model;
}

Utworzmy pliki tsv

In [ ]:
CreateTsvFiles();

Utwórzmy nasz model

In [ ]:
var model = GenerateModel(mlContext);

Sprawdźmy czy działa

In [ ]:
foreach (var file in Directory.GetFiles(_testFolder))
{
    var filename = Path.GetFileName(file);
    File.Copy(file, $"{_imagesFolder}\\{filename}", true);
    ClassifyImage(mlContext, model, filename);
}


[hammer.jpg] jest [hammer] z wynikiem [0.5069281] 
[screwdriver.jpg] jest [screwdriver] z wynikiem [0.66097254] 
[wrench.jpg] jest [wrench] z wynikiem [0.68434316] 
